In [9]:
import numpy as np
import pandas as pd
import requests
import time, random
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
import math

In [10]:
df = pd.read_excel('./딥러닝/옥션결과/seoul_827.xlsx',sheet_name = 'seoul_827 (1)')
df = df.drop('Unnamed: 0', 1)

In [11]:
df['BORN_YEAR'] = 1

In [12]:
def preprocessing(df):    
    
    #max size
    conditions = [
    (df['size_length'] > df['size_width']),
    (df['size_length'] < df['size_width']),
    (df['size_length'] == df['size_width']),
    ]
    choices = [df['size_length'], df['size_width'], df['size_width']]
    df['max_size'] = np.select(conditions, choices, default='black')
    df['max_size'] = df['max_size'].astype(float)    
    
    
    #canvas_size
    conditions = [    
     (df['max_size']<=20),
     (df['max_size']>20) & (df['max_size']<=25.5), (df['max_size']>25.5) & (df['max_size']<=30), (df['max_size']>30) & (df['max_size']<=34),
     (df['max_size']>34) & (df['max_size']<=38), (df['max_size']>38) & (df['max_size']<=43), (df['max_size']>43) & (df['max_size']<=50.5),
     (df['max_size']>50.5) & (df['max_size']<=58), (df['max_size']>58) & (df['max_size']<=63), (df['max_size']>63) & (df['max_size']<=69),
     (df['max_size']>69) & (df['max_size']<=77), (df['max_size']>77) & (df['max_size']<=88), (df['max_size']>88) & (df['max_size']<=97),
     (df['max_size']>97) & (df['max_size']<=110), (df['max_size']>110) & (df['max_size']<=125), (df['max_size']>125) & (df['max_size']<=139),
     (df['max_size']>139) & (df['max_size']<=154), (df['max_size']>154) & (df['max_size']<=163), (df['max_size']>163) & (df['max_size']<=195),
     (df['max_size']>195) & (df['max_size']<=240), (df['max_size']>240) & (df['max_size']<=270), (df['max_size']>270) & (df['max_size']<=300),
     (df['max_size']>300)
        ]
    choices = [1,2,3,4,5,6,8,10,12,15,20,25,30,40,50,60,80,100,120,150,200,300,500]
    df['canvas_size'] = np.select(conditions, choices, default='black').astype(float)

    
    #artwork_type
    conditions = [    
    (df['mix_cd'].str.contains('height')) & (df['mix_size'] != 0),
    (df['medium_eng'].str.contains("photo")) | (df['medium_eng'].str.contains("Photo")),
    (df['medium_eng'].str.contains("lithograph")) | (df['medium_eng'].str.contains("offset print")) | (df['medium_eng'].str.contains("lenticular")) | (df['medium_eng'].str.contains("silk screen")) | (df['medium_eng'].str.contains("silkscreen")) | 
    (df['medium_eng'].str.contains("mixed media")) | (df['medium_eng'].str.contains("drypoint")) | (df['medium_eng'].str.contains("serigraph")) | (df['medium_eng'].str.contains("etching")) | (df['medium_eng'].str.contains("litho")) | 
    (df['medium_eng'].str.contains("woodcut")) | (df['medium_eng'].str.contains("mezzotint")) | (df['medium_eng'].str.contains("aquatint")) | (df['medium_eng'].str.contains("collagraph")) | (df['medium_eng'].str.contains("monotype")) | 
    (df['medium_eng'].str.contains("monoprint")) | (df['medium_eng'].str.contains("mixograph")),
    (df['medium_eng'].str.contains("paper")) | (df['medium_eng'].str.contains("Paper")),
    (df['medium_eng'].str.contains("ink")) | (df['medium_eng'].str.contains("Ink")) | (df['medium_eng'].str.contains("oil")) | (df['medium_eng'].str.contains("Oil")) |
    (df['medium_eng'].str.contains("acrylic")) | (df['medium_eng'].str.contains("Acrylic")) | (df['medium_eng'].str.contains("Color")) | (df['medium_eng'].str.contains("color"))     
    ]
    choices = ['sculpture', 'photographs', 'print', 'work on paper', 'paintings']
    df['artwork_type'] = np.select(conditions, choices, default='other')

    
    #half_year
    conditions = [    
    (df['auction_date'].dt.month > 6) 
    ]
    choices = ['2nd half']
    df['half_year'] = np.select(conditions, choices, default='1st half')
    
    
    #convert_inch
    df['size_length_inch'] = df['size_length']*0.393701
    df['size_width_inch'] = df['size_width']*0.393701
    df['mix_size_inch'] = df['mix_size']*0.393701
    
    
    #canvas_size_money    
    df['canvas_size_money'] = np.where((df['artwork_type'] == 'work on paper')|(df['artwork_type'] == 'paintings'), df['money']/df['canvas_size'], np.nan)
    df['canvas_size_money'] = df['canvas_size_money'].astype(float)
    
    #name+born
    df['artist_name_kor_born'] = df['artist_name_kor'].astype(str) + ' (' + df['BORN_YEAR'].astype(str) +')'
    
    return df

In [13]:
def index(df):
    #평균 낙찰가
    avg_money = df.groupby(['artist_name_kor_born'])[['money']].mean()
    avg_money['avg_rank'] = avg_money['money'].rank(method = 'dense', ascending=False)
    avg_money = avg_money.sort_values(by=['avg_rank'], ascending=True)

    #호당 낙찰가
    canvas_avg_money = df.groupby(['artist_name_kor_born'])[['canvas_size_money']].mean()
    canvas_avg_money['canvas_avg_rank'] = canvas_avg_money['canvas_size_money'].rank(method = 'dense', ascending=False)
    canvas_avg_money = canvas_avg_money.sort_values(by=['canvas_avg_rank'], ascending=True)

    #최고 낙찰가
    max_money = df.groupby(['artist_name_kor_born'])[['money']].max()
    max_money['max_rank'] = max_money['money'].rank(method = 'dense', ascending=False)
    max_money = max_money.sort_values(by=['max_rank'], ascending=True)

    #총 판매가
    sum_money = df.groupby(['artist_name_kor_born'])[['money']].sum()
    sum_money['sum_rank'] = sum_money['money'].rank(method = 'dense', ascending=False)
    sum_money = sum_money.sort_values(by=['sum_rank'], ascending=True)

    #출품수
    count_money = df.groupby(['artist_name_kor_born'])[['artist_name_kor_born']].count()
    count_money['count_rank'] = count_money['artist_name_kor_born'].rank(method = 'dense', ascending=False)
    count_money = count_money.sort_values(by=['count_rank'], ascending=True)

    #최근 6개월 상승율
    df_6month = df[(df['auction_date'] < datetime.today().strftime('%Y-%m-%d')) & (df['auction_date'] >= (datetime.today() - relativedelta(months=+6)).strftime('%Y-%m-%d'))]
    df_12month = df[(df['auction_date'] < (datetime.today() - relativedelta(months=+6)).strftime('%Y-%m-%d')) & (df['auction_date'] >= (datetime.today() - relativedelta(months=+12)).strftime('%Y-%m-%d'))]

    df_6month_avg_money = df_6month.groupby(['artist_name_kor_born'])[['money']].mean()
    df_12month_avg_money = df_12month.groupby(['artist_name_kor_born'])[['money']].mean()

    recent_increased_rate = pd.merge(df_6month_avg_money, df_12month_avg_money, left_index = True, right_index = True, how = 'inner')
    recent_increased_rate = recent_increased_rate.dropna(axis = 0)

    recent_increased_rate['increased_rate'] = (recent_increased_rate['money_x'] - recent_increased_rate['money_y'])/recent_increased_rate['money_y']*100
    recent_increased_rate = recent_increased_rate[['increased_rate']]
    recent_increased_rate['increased_rate'] = round(recent_increased_rate['increased_rate'],1).astype(str) + '%'

    #전체 랭크
    total_index = pd.merge(avg_money, canvas_avg_money, left_index = True, right_index = True, how = 'left')
    total_index = pd.merge(total_index, max_money, left_index = True, right_index = True, how = 'left')
    total_index = pd.merge(total_index, sum_money, left_index = True, right_index = True, how = 'left')
    total_index = pd.merge(total_index, count_money, left_index = True, right_index = True, how = 'left')
    total_index = pd.merge(total_index, recent_increased_rate, left_index = True, right_index = True, how = 'left')

    total_index = total_index[['increased_rate','avg_rank','canvas_avg_rank','max_rank','sum_rank','count_rank']]
    total_index['total_sum'] = total_index['avg_rank']+ total_index['canvas_avg_rank']+total_index['max_rank']+total_index['sum_rank']+total_index['count_rank']
    total_index['total_rank'] = total_index['total_sum'].rank(method = 'dense', ascending=True)
    total_index = total_index.sort_values(by=['total_rank'], ascending=True)
    
    return avg_money, canvas_avg_money, max_money, sum_money, count_money, recent_increased_rate, total_index

In [14]:
df = preprocessing(df)
avg_money, canvas_avg_money, max_money, sum_money, count_money, recent_increased_rate, total_index = index(df)

In [15]:
total_index

,increased_rate,avg_rank,canvas_avg_rank,max_rank,sum_rank,count_rank,total_sum,total_rank
artist_name_kor_born,,,,,,,,
박수근 (1),4.5%,12.0,3.0,2.0,4.0,27.0,48.0,1.0
야요이 쿠사마 (1),184.8%,35.0,13.0,5.0,5.0,8.0,66.0,2.0
김환기 (1),15.1%,17.0,46.0,3.0,2.0,6.0,74.0,3.0
이중섭 (1),188.1%,5.0,5.0,1.0,6.0,73.0,90.0,4.0
천경자 (1),-32.1%,32.0,21.0,15.0,8.0,18.0,94.0,5.0
...,...,...,...,...,...,...,...,...
황리쩐 (1),NaN,NaN,NaN,NaN,1162.0,142.0,NaN,NaN
황빈홍 외 (1),NaN,NaN,NaN,NaN,1162.0,142.0,NaN,NaN
황창하 (1),NaN,NaN,NaN,NaN,1162.0,142.0,NaN,NaN


In [ ]:
import FinanceDataReader as fdr
usd = fdr.DataReader('USD/KRW', '1900')
usd = fdr.DataReader('JPY/KRW', '1900')
usd = fdr.DataReader('EUR/KRW', '1900')
usd = fdr.DataReader('GBP/KRW', '1900')